In [21]:
import pandas as pd

data = pd.read_excel("publicaciones/Publicaciones con Abstract ESPOL.xlsx")

In [22]:
coauthors =[]

for author in data['Authors']:
    author = str(author)
    if ";" in author:
        coauthors.append(author.split(";"))
    else:
        coauthors.append(author.split(".,"))
data['Keywords']

0      Additive manufacturing, Biomechanics, Mechanic...
1      3D additive printing, Biomechanics, Computing ...
2                                                    NaN
3      Damping control, Hydraulic pumping system, Osc...
4         Alarm systems, GSM, Messaging systems, NIOS II
                             ...                        
617              C. elegans, Image analysis, Recognition
618                           Filovirus, IFN, VP24, VP35
619                                                  NaN
620                                                  NaN
621    Heterologous promoters, Luciferase expression,...
Name: Keywords, Length: 622, dtype: object

In [23]:
def formating(author):
    author = author.strip()
    if not (author.endswith(".")):
        author = author+"."
    return author

dic = {}
authors = {}
for i,publ in enumerate(coauthors):
    
    keywords = str(data['Keywords'][i]).split(",")
    
    for offset,author1 in enumerate(publ):
        author1 = formating(author1)
        if author1 not in authors.keys():
            authors[author1] = set()
        for keyword in keywords:
            keyword = keyword.strip()
            if(keyword!='nan'):
                authors[author1].add(keyword)
        
        offset+=1
        author1 = formating(author1)
        for author2 in publ[offset:]:
            author2 = formating(author2)
            
            key = author1+";"+author2
            altkey = author2+";"+author1
            year = data['Year'][i]
            if year>=2015 and year<=2019:
                if key in dic.keys():
                    dic[key]['n']+=1

                    if year < dic[key]['firstColab']:
                        dic[key]['firstColab'] = year
                elif altkey in dic.keys():
                    dic[altkey]['n']+=1
                    if year < dic[altkey]['firstColab']:
                        dic[altkey]['firstColab'] = year
                else:
                    dic[key] = {"n":1,"firstColab":year}


In [24]:
with open("data/grafo_completo/coauthors-edges-ESPOL.csv","w",encoding="utf-8") as f:
    f.write("Source;Target;Weight;Year\n")
    for key, value in dic.items():
        ath = key.split(";")
        f.write(ath[0]+";"+ath[1]+";"+str(value['n'])+";"+str(int(value['firstColab']))+"\n")

with open("data/grafo_completo/coauthors-nodes-ESPOL.csv","w",encoding="utf-8") as g:
    g.write("ID;Label;Keywords\n")
    for auth,keyword in authors.items():
        g.write(auth+";"+auth+";"+",".join(keyword)+"\n")       
        

In [25]:
import networkx as nx

def connected_component_subgraphs(G):
    for c in nx.connected_components(G):
        yield G.subgraph(c)

coau_df = pd.read_csv("data/grafo_completo/coauthors-edges-ESPOL.csv",delimiter=";")

G = nx.from_pandas_edgelist(coau_df,"Source","Target",['Weight','Year'],create_using=nx.Graph())
G = sorted(connected_component_subgraphs(G), key=len, reverse=True)[0]
nodosCG = list(G.nodes)

with open("data/componente_gigante/coauthors-edgesCG-ESPOL.csv","w",encoding="utf-8") as f:
    f.write("Source;Target;Weight;Year\n")
    for key, value in dic.items():
        ath = key.split(";")
        if ath[0] in nodosCG:
            f.write(ath[0]+";"+ath[1]+";"+str(value['n'])+";"+str(int(value['firstColab']))+"\n")

with open("data/componente_gigante/coauthors-nodesCG-ESPOL.csv","w",encoding="utf-8") as g:
    g.write("ID;Label;Keywords\n")
    for auth,keyword in authors.items():
        if auth in nodosCG:
            g.write(auth+";"+auth+";"+",".join(keyword)+"\n")       
        

In [26]:
len(G.edges)

1175